https://www.kaggle.com/c/competitive-data-science-final-project

In [48]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline 

import utility.utility as ut
from preprocessing.createSets import *
from preprocessing.Sets import Sets


########### switches ################################

saveName='Oct11-2018_1'
saveFolder = saveName
saveplots=False
createSubmit = False
if not os.path.exists(saveFolder) and (saveplots or createSubmit): 
    print 'creating folder:',os.getcwd()+'/'+saveFolder
    os.mkdir(saveFolder)
if(saveplots): print "SAVING PLOTS!"
if(createSubmit): print "WILL CREATE SUBMIT FILE!"


############# Options/Args #########################

#columns to keep
col_to_keep = ['shop_id',
               'item_id',
               #'item_price',
               'item_category_id',
               'item_cnt_day',
              ]

groupby_list = ['shop_id','item_id']

agg_dict = {
            #'item_price':'mean',
            'item_category_id':'mean',
            'item_cnt_day':'sum',
            }

agg_targ = {'item_cnt_day':'sum'}

col_targets=['shop_item_cnt_month',
             'shop_cnt_month',
             'item_cnt_month',
             'shop_item_cnt_month_diff(0-1)',
             'shop_cnt_month_diff(0-1)',
             'item_cnt_month_diff(0-1)']

target = 'shop_item_cnt_month'
# target = 'shop_item_cnt_month_diff(0-1)'

opt = {'lag_length':3,
            'diff':True,
            'diffRel':False,
            'item_cat_count_feat':False,
            'target':target,
            'col_to_keep':col_to_keep,
            'groupby_list':groupby_list,
            'agg_dict':agg_dict,
            'agg_targ':agg_targ,
            'col_targets':col_targets,            
            } 
    
clipTarget = True
lowerClip = 0
upperClip = 20
if target=='shop_item_cnt_month_diff(0-1)':
    lowerClip = -20
    upperClip = 20    

########################################################


# create train/val/test set

In [49]:
Sets = Sets(**opt)

lag_length: 3
diff: True
diffRel: False
item_cat_count_feat : False
target: shop_item_cnt_month


In [50]:
Sets.splitDataByYear()
data = Sets.getData()

Splitting sales_train to sales_train_2013, sales_train_2014, and sales_train_2015.
Retrieving latest (preprocessed) data


# Check for duplicates

In [38]:
train = data['sales_train']
train[train.duplicated()]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M
1496765,2014-03-23,14,21,3423,999.0,1.0,23,2014,3,2014_3
1671873,2014-05-01,16,50,3423,999.0,1.0,23,2014,5,2014_5
1435365,2014-02-23,13,50,3423,999.0,1.0,23,2014,2,2014_2
1866340,2014-07-12,18,25,3423,999.0,1.0,23,2014,7,2014_7
76962,2013-01-05,0,54,20130,149.0,1.0,40,2013,1,2013_1
2198564,2014-12-31,23,42,21619,499.0,1.0,37,2014,12,2014_12


In [39]:
train[(train['date']=='2014-03-23')&(train['shop_id']==21)&(train['item_id']==3423)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M
1496766,2014-03-23,14,21,3423,999.0,1.0,23,2014,3,2014_3
1496765,2014-03-23,14,21,3423,999.0,1.0,23,2014,3,2014_3


In [40]:
train[(train['date']=='2014-05-01 	')&(train['shop_id']==50)&(train['item_id']==3423)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M
1671872,2014-05-01,16,50,3423,999.0,1.0,23,2014,5,2014_5
1671873,2014-05-01,16,50,3423,999.0,1.0,23,2014,5,2014_5


In [41]:
train = train.drop_duplicates(keep='first')

In [42]:
train[(train['date']=='2014-03-23')&(train['shop_id']==21)&(train['item_id']==3423)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M
1496766,2014-03-23,14,21,3423,999.0,1.0,23,2014,3,2014_3


In [43]:
train[train.duplicated()]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M


In [44]:
test = data['test']
test[test.duplicated()]

,ID,shop_id,item_id


In [52]:
Sets.checkDuplicates()

checking for duplicates in sales_train
Found 6 duplicates in sales_train
Kept first, removed duplicates
checking for duplicates in test
Found no duplicates in test


In [53]:
data['sales_train'][data['sales_train'].duplicated()]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M
